<a href="https://colab.research.google.com/github/peterbabulik/ETA/blob/main/BornRule.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install qiskit qiskit_aer -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 52.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 95.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 71.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 3.7 MB/s eta 0:00:00


To translate the equation $P(\psi_1 \to \psi_2) = |\langle\psi_1|\psi_2\rangle|^2$ into a quantum circuit, we generally use the **Compute-Uncompute** method (also known as the Inversion Test).

### The Logic
1.  **Preparation**: Let $U_1$ be the circuit that prepares $|\psi_1\rangle$ from $|0\rangle$, and $U_2$ be the circuit that prepares $|\psi_2\rangle$.
2.  **Transition**: To measure the overlap (fidelity), we prepare $|\psi_2\rangle$ (apply $U_2$), then try to "uncompute" $|\psi_1\rangle$ (apply $U_1^\dagger$).
3.  **Measurement**: We measure the probability of returning to the ground state $|0\rangle$.
    *   If the states are identical, $U_1^\dagger U_2 = I$, and we get $|0\rangle$ with probability 1.
    *   If they are orthogonal, we get $|0\rangle$ with probability 0.
    *   The probability of measuring $0$ is exactly $|\langle\psi_1|\psi_2\rangle|^2$.

Here is the Quiskit implementation using the current Qiskit SDK (v1.x style).

In [ ]:
import numpy as np
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram

def calculate_transition_probability(theta1, theta2):
    """
    Calculates P(ψ1 -> ψ2) = |<ψ1|ψ2>|^2 using the compute-uncompute method.
    We use Ry rotations to create simple states to visualize the angle/distance.
    """

    # 1. Setup the circuit
    # We need 1 qubit (or N qubits depending on state complexity)
    qc = QuantumCircuit(1, 1)

    # -------------------------------------------------------
    # STEP A: Prepare state |ψ2> (Apply U2)
    # Let's define |ψ> = Ry(theta)|0>
    # -------------------------------------------------------
    qc.ry(theta2, 0)
    qc.barrier() # Visual separator

    # -------------------------------------------------------
    # STEP B: Apply Inverse of |ψ1> (Apply U1†)
    # Since U1 is Ry(theta1), U1† is Ry(-theta1)
    # -------------------------------------------------------
    qc.ry(-theta1, 0)
    qc.barrier()

    # -------------------------------------------------------
    # STEP C: Measure in the computational basis
    # P(0) corresponds to |<ψ1|ψ2>|^2
    # -------------------------------------------------------
    qc.measure(0, 0)

    # 2. Execute the circuit
    simulator = AerSimulator()
    compiled_circuit = transpile(qc, simulator)
    job = simulator.run(compiled_circuit, shots=10000)
    result = job.result()
    counts = result.get_counts()

    # 3. Calculate Probability P(0)
    # If '0' is not in counts, the probability is 0
    count_0 = counts.get('0', 0)
    measured_prob = count_0 / 10000

    return measured_prob, qc

# --- Example Usage ---

# Define two angles (The 'distance' in Hilbert space relates to the difference)
angle_psi1 = 0.0          # State |0>
angle_psi2 = np.pi / 3    # State rotated by 60 degrees

prob, circuit = calculate_transition_probability(angle_psi1, angle_psi2)

# Theoretical Calculation
# For single qubits rotated by Ry: <ψ1|ψ2> = cos((θ1 - θ2)/2)
# Therefore P = cos^2((θ1 - θ2)/2)
fubini_study_distance = (angle_psi1 - angle_psi2) / 2
theoretical_prob = np.cos(fubini_study_distance)**2

print(f"--- Results ---")
print(f"Angle difference (d_FS approx): {fubini_study_distance:.4f}")
print(f"Measured Probability P(ψ1 -> ψ2): {prob:.4f}")
print(f"Theoretical cos²(d_FS):           {theoretical_prob:.4f}")

# Draw the circuit
print("\n--- Circuit Diagram ---")
print(circuit)

--- Results ---
Angle difference (d_FS approx): -0.5236
Measured Probability P(ψ1 -> ψ2): 0.7505
Theoretical cos²(d_FS):           0.7500

--- Circuit Diagram ---
     ┌─────────┐ ░ ┌───────┐ ░ ┌─┐
  q: ┤ Ry(π/3) ├─░─┤ Ry(0) ├─░─┤M├
     └─────────┘ ░ └───────┘ ░ └╥┘
c: 1/═══════════════════════════╩═
                                0 


### Explanation of the Output

1.  **$|\psi_1\rangle$ and $|\psi_2\rangle$**: In the code, these are defined by rotation angles.
2.  **$U_2$**: Rotates the qubit to state 2.
3.  **$U_1^\dagger$**: Rotates the qubit "backwards" by the amount required to create state 1.
4.  **Measurement**:
    *   If the result is **0**, the transition was "successful" (the states overlapped).
    *   The count of zeros divided by total shots gives you **$P = \cos^2(d_{FS})$**.

### Alternative: The Swap Test
If you cannot construct $U^\dagger$ (the inverse circuit) but you have two separate quantum registers containing $|\psi_1\rangle$ and $|\psi_2\rangle$, you would use the **Swap Test**.

*   **Circuit**: 1 Ancilla qubit, Register 1 ($|\psi_1\rangle$), Register 2 ($|\psi_2\rangle$).
*   **Operations**: `H(ancilla)` $\to$ `CSWAP(ancilla, reg1, reg2)` $\to$ `H(ancilla)` $\to$ `Measure(ancilla)`.
*   **Result**: $P(0) = \frac{1}{2} + \frac{1}{2}|\langle\psi_1|\psi_2\rangle|^2$.
*   *Note: This requires math post-processing to isolate the squared overlap.*